In [1]:
import numpy as np
import os
import json
import pandas as pd
import time
from tqdm import tqdm
import gc
import mmap
from scipy import spatial
import heapq

In [72]:

class TracksArchive:
    def __init__(self, coll_dir, archive_path = 'tracks_archive.txt'):
        self.coll_dir = coll_dir
        self.archive_path = archive_path
        self.top_tracks = []
        self.top_artists = []
        self.top_playlists = []
    
    def make_top_list(self):
        with open('../stats.txt','r') as f:
            top_cat = None
            lines = f.readlines()
            for line in lines:
                if line == '\n':
                    top_cat = None
                
                if top_cat == 'P':
                    playlist = line.strip().split()[1:]
                    playlist = ' '.join(playlist)
                    #print('p',playlist)
                    self.top_playlists.append(playlist)
                elif top_cat == 'T':
                    track = line.strip().split()[1:]
                    track = ' '.join(track)
                    #print('t',track)
                    self.top_tracks.append(track)
                elif top_cat == 'A':
                    artist = line.strip().split()[1:]
                    artist = ' '.join(artist)
                    #print('a',artist)
                    self.top_artists.append(artist)
                    
                if line == 'top playlist titles\n':
                    top_cat = 'P'
                elif line == 'top tracks\n':
                    top_cat = 'T'
                elif line == 'top artists\n':
                    top_cat = 'A'
                    
        print('fetched details')
        return True
            

    def save_top_tracks(self, limit_track_save = 500):
        #limit_track_save = 50  #Later 500
        imp_track_count = 0
        limit_file_read_to = 1
        self.make_top_list()
        print('Creating songs archive')
        file_count = 0
        coll_dir = self.coll_dir
        
        #Empty the file
        f = open(self.archive_path, '+w')
        f.close()
        
        tracks_set = set()
        
        for filename in os.listdir(coll_dir)[:limit_file_read_to]:
            file_count += 1
            #print('For File:', filename)
            with open(os.path.join(coll_dir, filename), 'r') as f:
                mm = mmap.mmap(f.fileno(),length = 0, access = mmap.ACCESS_READ)
                playlists = json.load(mm)['playlists']
                # print('total playlists:', len(playlists))
                for playlist in playlists[:]:
                    imp_pl = False
                    if playlist['name'] in self.top_playlists:
                        print('its a top pl', playlist['name'])
                        imp_pl = True
                    tracks = playlist['tracks']
                    # print('\tTotal Tracks:', len(tracks))
                    for track in tracks:
                        imp_track = False
                        if track['track_name'] in self.top_tracks or track['artist_name'] in self.top_artists or imp_pl:
                            imp_track = True
                            #print('imp track found')
                            #print(track['track_uri'], track['track_name'], track['artist_name'])
                            if track['track_uri'] not in tracks_set:
                                imp_track_count +=1
                                tracks_set = tracks_set | set([track['track_uri']])
                                content = track['track_uri'] + '\n'
                                with open(self.archive_path, '+a') as file:
                                    file.write(content)
                        if imp_track_count>=limit_track_save:
                            break
                    if imp_track_count>=limit_track_save:
                        break
                mm.close()
            if imp_track_count>=limit_track_save:
                break
        print('tracks stored:',imp_track_count)
        return tracks_set


collection_dir = '/media/harsh/Common/IITD/COL764-IR/Project/spotify_million_playlist_dataset/data/'

In [73]:
archive = TracksArchive(collection_dir)

In [76]:
_ = archive.save_top_tracks(500)

fetched details
Creating songs archive
its a top pl disney
its a top pl sleep
its a top pl rap
its a top pl vibes
tracks stored: 500


['HUMBLE. by Kendrick Lamar',
 'One Dance by Drake',
 'Broccoli (feat. Lil Yachty) by DRAM',
 'Closer by The Chainsmokers',
 'Congratulations by Post Malone',
 'Caroline by Aminé',
 'iSpy (feat. Lil Yachty) by KYLE',
 'Bad and Boujee (feat. Lil Uzi Vert) by Migos',
 'Location by Khalid',
 'XO TOUR Llif3 by Lil Uzi Vert',
 'Bounce Back by Big Sean',
 'Ignition - Remix by R. Kelly',
 'No Role Modelz by J. Cole',
 'Mask Off by Future',
 'No Problem (feat. Lil Wayne & 2 Chainz) by Chance The Rapper',
 "I'm the One by DJ Khaled",
 'Jumpman by Drake',
 'goosebumps by Travis Scott',
 'Fake Love by Drake',
 'Despacito - Remix by Luis Fonsi']